In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [10]:
class Network(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3) # i/p 28x28x1  10x3x3x1 >> o/p 26x26x10 RF 1
    self.conv2 = nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3) # i/p 26x26x10  10x3x3x10 >>o/p 24x24x10 RF 3
    self.conv3 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=3) # i/p 24x24x10  10x3x3x10x20 >>o/p 22x22x20 RF 5
    self.pool1 = nn.MaxPool2d(2, 2)  #i/p 22x22x20 >> o/p 11x11x20 RF 10
    self.batch1= nn.BatchNorm2d(20)  #added recently
    self.conv4 = nn.Conv2d(in_channels=20, out_channels=20, kernel_size=3) # i/p 11x11x20  10x3x3x10x20x20  >>o/p 9x9x20 RF 12
    self.conv5 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3) # i/p 9x9x20 >> 10x3x3x10x20x20x30 >>o/p 7x7x30 RF 14 
    #self.gpool3 = nn.AvgPool2d(2,2) # added recently   
    self.pool2 = nn.MaxPool2d(2, 2)  # ip 7x7x30  >> o/p 3x3x30 RF 28 
    self.batch2 = nn.BatchNorm2d(30)    # added recently
    self.conv6 = nn.Conv2d(in_channels=30, out_channels=10  , kernel_size=3) # i/p 7x7x30 >>  10x3x3x10x20x20x30x10  >> o/p 5x5x10  RF 30
    
     

  def forward(self, x):
    x = self.pool1(F.relu(self.conv3(F.relu(self.conv2(F.relu(self.conv1(x)))))))   # relu layer drops all negative values and propagates positive values as it is
    x = self.pool2(F.relu(self.conv5(F.relu(self.conv4( self.batch1(x))))))    
    x = self.conv6(self.batch2(x))  
    # x = self.gpool3(x)   # added recently
    x = x.view(-1, 10)
    return F.log_softmax(x)

In [11]:
!pip install torchsummary    # Run the shell command from the script , it installs torchsummary package from shell
from torchsummary import summary
use_cuda = torch.cuda.is_available() # checks for the availability of GPU, if available returns True else False
# print(use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")  # Device stores "cuda" if GPU is available or 'cpu' if GPU is not available
print(device)
model = Network().to(device)                # Moves the model to GPU
summary(model, input_size=(1, 28, 28))  # Input Image size is 28x28X1, 1 here specifies only 1 channel of the input i.e, gray scale image

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
            Conv2d-2           [-1, 10, 24, 24]             910
            Conv2d-3           [-1, 20, 22, 22]           1,820
         MaxPool2d-4           [-1, 20, 11, 11]               0
       BatchNorm2d-5           [-1, 20, 11, 11]              40
            Conv2d-6             [-1, 20, 9, 9]           3,620
            Conv2d-7             [-1, 30, 7, 7]           5,430
         MaxPool2d-8             [-1, 30, 3, 3]               0
       BatchNorm2d-9             [-1, 30, 3, 3]              60
           Conv2d-10             [-1, 10, 1, 1]           2,710
Total params: 14,690
Trainable params: 14,690
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.23
Params size (MB): 0.06
Estimate

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [12]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [13]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:

model = Network().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS=19
for epoch in range(EPOCHS):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0477, Accuracy: 9861/10000 (98.61%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0376, Accuracy: 9888/10000 (98.88%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0216, Accuracy: 9939/10000 (99.39%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0208, Accuracy: 9941/10000 (99.41%)



loss=0.004092533607035875 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.30it/s]



Test set: Average loss: 0.0212, Accuracy: 9944/10000 (99.44%)

